In [1]:
import warnings
warnings.filterwarnings('ignore')

### 1. Preparation

#### 1.1 Import 

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
# from torchvision.models.resnet import resnet18
# from torchvision.models.vgg import vgg11
from tqdm import tqdm
import numpy as np

In [3]:
import errno
import os
import os.path as osp
import shutil
from collections import OrderedDict
import time

#### 1.2 Define transform and load data set

In [4]:
train_transform = transforms.Compose([
        transforms.Pad(4, padding_mode='reflect'),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
        transforms.ToTensor(),
        transforms.Normalize(
            np.array([125.3, 123.0, 113.9]) / 255.0,
            np.array([63.0, 62.1, 66.7]) / 255.0),
    ])
val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            np.array([125.3, 123.0, 113.9]) / 255.0,
            np.array([63.0, 62.1, 66.7]) / 255.0),
    ])

In [5]:
train_dataset = CIFAR100(root='cifar100', train=True, download=True, transform=train_transform)
valid_dataset = CIFAR100(root='cifar100', train=False, download=True, transform=val_transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_loader = DataLoader(train_dataset,
                              batch_size=128,
                              shuffle=True)
valid_loader = DataLoader(valid_dataset,
                            batch_size=128)

#### 1.3 Define network

In [7]:
# resnet = resnet18(pretrained=False, num_classes=100)
# vgg = vgg11(pretrained=False, num_classes=100)

#### 1.4 Define loss function

In [8]:
criterion = nn.CrossEntropyLoss()

#### 1.5 Define the training and test process

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    #滑动平均
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    #topk准确率
    #预测结果前k个中出现的正确结果的次数
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [11]:
def mkdir_if_missing(directory):
    #创建文件夹，如果这个文件夹不存在的话
    if not osp.exists(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

In [12]:
def save_checkpoint(state, is_best=False, fpath=''):
    if len(osp.dirname(fpath)) != 0:
        mkdir_if_missing(osp.dirname(fpath))
    torch.save(state, fpath)
    if is_best:
        shutil.copy(fpath, osp.join(osp.dirname(fpath), 'best_model.pth.tar'))

In [13]:
def adjust_learning_rate(optimizer, epoch, initial_lr):
    """decrease the learning rate at 100 and 150 epoch"""
    lr = initial_lr
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [14]:
def warp_tqdm(data_loader, disable_tqdm):
    #进度条打印
    if disable_tqdm:
        tqdm_loader = data_loader
    else:
        tqdm_loader = tqdm(data_loader, ncols=0)
    return tqdm_loader

In [15]:
def train(train_loader, model, criterion, optimizer, epoch):
    #每个epoch的优化过程
    losses = AverageMeter()

    # switch to train mode
    model.train()

    for input, target in warp_tqdm(train_loader, True):


        input = input.cuda()
        target = target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        losses.update(loss.item(), input.size(0))

    log = 'Epoch:{0}\tLoss: {loss.avg:.4f}\t'.format(epoch, loss=losses)
    return log

In [16]:
def test(test_loader, model, criterion):
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    for input, target in test_loader:

        # compute output
        with torch.no_grad():
            input = input.cuda()
            target = target.cuda()
            output = model(input)

        # measure accuracy and record loss
        acc1 = accuracy(output.data, target)[0]
        top1.update(acc1.item(), input.size(0))

        # measure elapsed time

    log = 'Test Acc@1: {top1.avg:.3f}'.format(top1=top1)

    return top1.avg, log

### 2. Train a VGG/ResNet on CIFAR-100

In [17]:
# ckpt = 'experiments/cifar100-vgg'
# num_epochs = 20
# model = vgg.cuda()
# train_loader = train_loader
# test_loader = valid_loader
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
# best_acc = 0
# for epoch in range(num_epochs):
#     adjust_learning_rate(optimizer, epoch, 0.1)
#     train_log = train(train_loader, model, criterion, optimizer, epoch)
#     acc, test_log = test(test_loader, model, criterion)
#     log = train_log + test_log
#     print(log)
#     is_best = acc > best_acc
#     best_acc = max(acc, best_acc)
#     if is_best:
#         save_checkpoint({'epoch':epoch,
#         'state_dict':model.state_dict(),
#         'acc': acc,
#         }, False, 'best_model.pth.tar')

In [18]:
ckpt = 'experiments/cifar100-resnet'
from net.resnet import resnet18
num_epochs = 200
model = resnet18()
model = model.cuda()
train_loader = train_loader
test_loader = valid_loader
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
best_acc = 0
for epoch in range(num_epochs):
    adjust_learning_rate(optimizer, epoch, 0.1)
    train_log = train(train_loader, model, criterion, optimizer, epoch)
    acc, test_log = test(test_loader, model, criterion)
    scheduler.step()
    log = train_log + test_log
    print(log)
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)
    if is_best:
        save_checkpoint({'epoch':epoch,
        'state_dict':model.state_dict(),
        'acc': acc,
        }, False, 'best_model.pth.tar')

Epoch:0	Loss: 3.8583	Test Acc@1: 14.830
Epoch:1	Loss: 3.1818	Test Acc@1: 28.210
Epoch:2	Loss: 2.6438	Test Acc@1: 34.100
Epoch:3	Loss: 2.2708	Test Acc@1: 38.120
Epoch:4	Loss: 2.0209	Test Acc@1: 41.180
Epoch:5	Loss: 1.8594	Test Acc@1: 44.400
Epoch:6	Loss: 1.7324	Test Acc@1: 48.650
Epoch:7	Loss: 1.6488	Test Acc@1: 47.330
Epoch:8	Loss: 1.5713	Test Acc@1: 52.250
Epoch:9	Loss: 1.5037	Test Acc@1: 51.740
Epoch:10	Loss: 1.4584	Test Acc@1: 52.680
Epoch:11	Loss: 1.4232	Test Acc@1: 53.640
Epoch:12	Loss: 1.3804	Test Acc@1: 54.320
Epoch:13	Loss: 1.3474	Test Acc@1: 53.350
Epoch:14	Loss: 1.3332	Test Acc@1: 53.610
Epoch:15	Loss: 1.2980	Test Acc@1: 51.560
Epoch:16	Loss: 1.2828	Test Acc@1: 53.110
Epoch:17	Loss: 1.2602	Test Acc@1: 53.330
Epoch:18	Loss: 1.2411	Test Acc@1: 56.370
Epoch:19	Loss: 1.2298	Test Acc@1: 54.180
Epoch:20	Loss: 1.2091	Test Acc@1: 56.570
Epoch:21	Loss: 1.1948	Test Acc@1: 57.200
Epoch:22	Loss: 1.1792	Test Acc@1: 54.430
Epoch:23	Loss: 1.1723	Test Acc@1: 56.250
Epoch:24	Loss: 1.1648	Test

In [ ]:
ckpt = 'experiments/cifar100-resnet'
from net.resnet import resnet18
from utils.mixup import mixup_train
num_epochs = 200
model = resnet18()
model = model.cuda()
train_loader = train_loader
test_loader = valid_loader
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
best_acc = 0
for epoch in range(num_epochs):
    adjust_learning_rate(optimizer, epoch, 0.1)
    train_log = mixup_train(train_loader, model, criterion, optimizer, epoch)
    acc, test_log = test(test_loader, model, criterion)
    scheduler.step()
    log = train_log + test_log
    print(log)
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)
    if is_best:
        save_checkpoint({'epoch':epoch,
        'state_dict':model.state_dict(),
        'acc': acc,
        }, False, 'best_model_mixup.pth.tar')

Epoch:0	Loss: 4.2144	Test Acc@1: 13.410
Epoch:1	Loss: 3.9004	Test Acc@1: 18.950
Epoch:2	Loss: 3.6564	Test Acc@1: 26.020
Epoch:3	Loss: 3.4454	Test Acc@1: 29.500
Epoch:4	Loss: 3.2514	Test Acc@1: 37.400
Epoch:5	Loss: 3.1567	Test Acc@1: 38.220
Epoch:6	Loss: 3.0594	Test Acc@1: 44.350
Epoch:7	Loss: 2.9979	Test Acc@1: 44.610
Epoch:8	Loss: 2.9544	Test Acc@1: 47.590
Epoch:9	Loss: 2.9104	Test Acc@1: 45.860
Epoch:10	Loss: 2.8797	Test Acc@1: 50.530
Epoch:11	Loss: 2.8385	Test Acc@1: 45.340
Epoch:12	Loss: 2.7911	Test Acc@1: 49.840
Epoch:13	Loss: 2.8631	Test Acc@1: 50.610
Epoch:14	Loss: 2.7527	Test Acc@1: 49.610
Epoch:15	Loss: 2.6976	Test Acc@1: 48.690
Epoch:16	Loss: 2.7478	Test Acc@1: 55.840
Epoch:17	Loss: 2.6854	Test Acc@1: 53.500
Epoch:18	Loss: 2.6543	Test Acc@1: 52.650
Epoch:19	Loss: 2.7000	Test Acc@1: 52.600
Epoch:20	Loss: 2.7359	Test Acc@1: 56.630
Epoch:21	Loss: 2.6685	Test Acc@1: 47.840
Epoch:22	Loss: 2.6703	Test Acc@1: 54.160
Epoch:23	Loss: 2.6225	Test Acc@1: 54.940
Epoch:24	Loss: 2.6414	Test

In [ ]:
ckpt = 'experiments/cifar100-resnet'
from net.resnet import resnet18
from utils.cutmix import cutmix_train
num_epochs = 200
model = resnet18()
model = model.cuda()
train_loader = train_loader
test_loader = valid_loader
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
best_acc = 0
for epoch in range(num_epochs):
    adjust_learning_rate(optimizer, epoch, 0.1)
    train_log = cutmix_train(train_loader, model, criterion, optimizer, epoch)
    acc, test_log = test(test_loader, model, criterion)
    scheduler.step()
    log = train_log + test_log
    print(log)
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)
    if is_best:
        save_checkpoint({'epoch':epoch,
        'state_dict':model.state_dict(),
        'acc': acc,
        }, False, 'best_model_cutmix.pth.tar')

Epoch:0	Loss: 4.3495	Test Acc@1: 10.050
Epoch:1	Loss: 4.1585	Test Acc@1: 13.900
Epoch:2	Loss: 4.0223	Test Acc@1: 20.220
Epoch:3	Loss: 3.8658	Test Acc@1: 25.000
Epoch:4	Loss: 3.7475	Test Acc@1: 27.970
Epoch:5	Loss: 3.5747	Test Acc@1: 32.770
Epoch:6	Loss: 3.5111	Test Acc@1: 34.850
Epoch:7	Loss: 3.4100	Test Acc@1: 37.640
Epoch:8	Loss: 3.3665	Test Acc@1: 33.860
Epoch:9	Loss: 3.2639	Test Acc@1: 37.720
Epoch:10	Loss: 3.2944	Test Acc@1: 39.580
Epoch:11	Loss: 3.2121	Test Acc@1: 43.450
Epoch:12	Loss: 3.2199	Test Acc@1: 43.710
Epoch:13	Loss: 3.2031	Test Acc@1: 39.850
Epoch:14	Loss: 3.2083	Test Acc@1: 41.100
Epoch:15	Loss: 3.1384	Test Acc@1: 44.240
Epoch:16	Loss: 3.1084	Test Acc@1: 46.220
Epoch:17	Loss: 3.0481	Test Acc@1: 49.240
Epoch:18	Loss: 3.0962	Test Acc@1: 50.030
Epoch:19	Loss: 3.0489	Test Acc@1: 42.140


In [ ]:
ckpt = 'experiments/cifar100-resnet'
from net.resnet import resnet18
from utils.cutout import cutout_train
num_epochs = 200
model = resnet18()
model = model.cuda()
train_loader = train_loader
test_loader = valid_loader
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
best_acc = 0
for epoch in range(num_epochs):
    adjust_learning_rate(optimizer, epoch, 0.1)
    train_log = cutout_train(train_loader, model, criterion, optimizer, epoch)
    acc, test_log = test(test_loader, model, criterion)
    scheduler.step()
    log = train_log + test_log
    print(log)
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)
    if is_best:
        save_checkpoint({'epoch':epoch,
        'state_dict':model.state_dict(),
        'acc': acc,
        }, False, 'best_model_cutout.pth.tar')

Epoch:0	Loss: 4.0402	Test Acc@1: 11.440
Epoch:1	Loss: 3.6340	Test Acc@1: 14.950
Epoch:2	Loss: 3.3421	Test Acc@1: 22.740
Epoch:3	Loss: 3.1401	Test Acc@1: 23.040
Epoch:4	Loss: 2.9621	Test Acc@1: 26.450
Epoch:5	Loss: 2.7842	Test Acc@1: 32.350
Epoch:6	Loss: 2.6508	Test Acc@1: 38.380
Epoch:7	Loss: 2.5202	Test Acc@1: 41.150
Epoch:8	Loss: 2.4383	Test Acc@1: 42.460
Epoch:9	Loss: 2.3333	Test Acc@1: 40.370
Epoch:10	Loss: 2.2727	Test Acc@1: 39.440
Epoch:11	Loss: 2.2335	Test Acc@1: 43.400
Epoch:12	Loss: 2.2066	Test Acc@1: 41.010
Epoch:13	Loss: 2.1160	Test Acc@1: 48.520
Epoch:14	Loss: 2.1084	Test Acc@1: 42.260
Epoch:15	Loss: 2.0905	Test Acc@1: 49.340
Epoch:16	Loss: 2.0541	Test Acc@1: 47.080
Epoch:17	Loss: 2.0652	Test Acc@1: 48.430
Epoch:18	Loss: 2.0231	Test Acc@1: 49.460
Epoch:19	Loss: 2.0214	Test Acc@1: 49.890
Epoch:20	Loss: 1.9714	Test Acc@1: 49.440
Epoch:21	Loss: 1.9611	Test Acc@1: 52.340
Epoch:22	Loss: 1.9502	Test Acc@1: 50.590
Epoch:23	Loss: 1.9938	Test Acc@1: 51.540
Epoch:24	Loss: 1.9740	Test